In [0]:
select sum(total_amount) as total_amount from dev.silver.sales_cleaned

In [0]:
WITH ProductDiscounts AS (
    SELECT 
        `product_id`, 
        `discount_amount`,
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY `discount_amount`) OVER (PARTITION BY `product_id`) AS Q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY `discount_amount`) OVER (PARTITION BY `product_id`) AS Q3
    FROM 
        `dev`.`silver`.`sales_cleaned`
),
ProductOutlierBounds AS (
    SELECT 
        `product_id`, 
        Q1, 
        Q3, 
        Q3 - Q1 AS IQR,
        Q1 - 1.5 * (Q3 - Q1) AS LowerBound,
        Q3 + 1.5 * (Q3 - Q1) AS UpperBound
    FROM 
        ProductDiscounts
    GROUP BY 
        `product_id`, Q1, Q3
)
SELECT 
    a.`product_id`, 
    a.`discount_amount`
FROM 
    `dev`.`silver`.`sales_cleaned` a
JOIN 
    ProductOutlierBounds b
ON 
    a.`product_id` = b.`product_id`
WHERE 
    a.`discount_amount` < b.LowerBound
    OR a.`discount_amount` > b.UpperBound